<a href="https://colab.research.google.com/github/Exabyte-io/api-examples/blob/dev/examples/material/api_interoperability_showcase.ipynb" target="_parent">
<img alt="Open in Google Colab" src="https://user-images.githubusercontent.com/20477508/128780728-491fea90-9b23-495f-a091-11681150db37.jpeg" width="150" border="0">
</a>

# Overview

This example was created as part of our [Advanced Topics Webinar](https://www.youtube.com/watch?v=psSFC409jSg) on February 19, 2021. This webinar focused on explaining our API in detail, and provided examples of many areas of its functionality.

In this notebook, we showcase a major advantage of APIs: interoperability. We begin by performing a query using the [Materials Project](https://materialsproject.org) API for all systems containing Iron and Oxygen. We then filter our results (for demonstraiton purposes, we keep only the first 10 materials found). Finally, we upload our results to the Mat3ra platform, where further calculations could be performed to characterize these materials.

# Complete Authorization Form and Initialize Settings

This will also determine environment and set all environment variables. We determine if we are using Jupyter Notebooks or Google Colab to run this tutorial.

If you are running this notebook from Google Colab, Colab takes ~1 min to execute the following cell.

ACCOUNT_ID and AUTH_TOKEN - Authentication parameters needed for when making requests to [Mat3ra.com's API Endpoints](https://docs.mat3ra.com/rest-api/endpoints/).

MATERIALS_PROJECT_API_KEY - Authentication parameter needed for when making requests to [Material Project's API](https://materialsproject.org/open)

ORGANIZATION_ID - Authentication parameter needed for when working with collaborative accounts https://docs.mat3ra.com/collaboration/organizations/overview/

> <span style="color: orange">**NOTE**</span>: If you are running this notebook from Jupyter, the variables ACCOUNT_ID, AUTH_TOKEN, MATERIALS_PROJECT_API_KEY, and ORGANIZATION_ID should be set in the file [settings.json](../settings.json) if you need to use these variables. To obtain API token parameters, please see the following link to the documentation explaining how to get them: https://docs.mat3ra.com/accounts/ui/preferences/api/

In [1]:
#@title Authorization Form
ACCOUNT_ID = "ACCOUNT_ID" #@param {type:"string"}
AUTH_TOKEN = "AUTH_TOKEN" #@param {type:"string"}
MATERIALS_PROJECT_API_KEY = "MATERIALS_PROJECT_API_KEY" #@param {type:"string"}
ORGANIZATION_ID  = "ORGANIZATION_ID" #@param {type:"string"}

import os
if "COLAB_JUPYTER_IP" in os.environ:
    os.environ.update(
        dict(
            ACCOUNT_ID=ACCOUNT_ID,
            AUTH_TOKEN=AUTH_TOKEN,
            MATERIALS_PROJECT_API_KEY=MATERIALS_PROJECT_API_KEY,
            ORGANIZATION_ID=ORGANIZATION_ID,
        )
    )

    !GIT_BRANCH="dev"; export GIT_BRANCH; curl -s "https://raw.githubusercontent.com/Exabyte-io/api-examples/${GIT_BRANCH}/scripts/env.sh" | bash

# Imports

In [2]:
from utils.settings import ENDPOINT_ARGS, MATERIALS_PROJECT_API_KEY
from utils.generic import display_JSON

import ase.io
from pymatgen.ext.matproj import MPRester

from exabyte_api_client.endpoints.materials import MaterialEndpoints

/Users/exabyte/.pyenv/versions/3.8.6/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Query the Materials Project

We begin by using the Materials Project API implemented in [PyMatGen](https://pymatgen.org/pymatgen.ext.matproj.html) to perform a query for all systems containing Iron and Oxygen.

# Query Materials Project for all systems containing Iron and Oxygen

In [3]:
materials_project_api = MPRester(MATERIALS_PROJECT_API_KEY)
iron_oxides_ids = materials_project_api.get_materials_ids("Fe-O")

print(iron_oxides_ids)

['mp-1271693', 'mp-136', 'mp-1271562', 'mp-1245108', 'mp-1096950', 'mp-1194030', 'mp-568345', 'mp-1271295', 'mp-13', 'mp-150', 'mp-540003', 'mp-1244911', 'mp-1181437', 'mp-705416', 'mp-764417', 'mp-1456', 'mp-32939', 'mp-1245078', 'mp-1103327', 'mp-763787', 'mp-1205415', 'mp-705555', 'mp-705588', 'mp-1062652', 'mp-705558', 'mp-715811', 'mvc-15135', 'mp-1068212', 'mp-1283030', 'mp-510252', 'mp-25332', 'mp-1224936', 'mp-849689', 'mp-1225007', 'mp-1244983', 'mp-1181411', 'mp-1195927', 'mp-530050', 'mvc-13234', 'mp-776135', 'mp-1279742', 'mp-715572', 'mp-1181334', 'mp-776606', 'mp-759037', 'mp-1245084', 'mp-1284941', 'mp-1188678', 'mp-1181657', 'mp-715614', 'mp-715558', 'mp-757565', 'mp-510080', 'mp-705424', 'mp-19770', 'mp-530048', 'mp-705414', 'mp-1095382', 'mp-1178247', 'mp-1194978', 'mp-650112', 'mp-505595', 'mp-756436', 'mp-557546', 'mvc-12205', 'mp-612405', 'mp-715276', 'mp-1178232', 'mvc-11541', 'mp-705753', 'mp-19306', 'mp-716814', 'mp-1291630', 'mp-715333', 'mp-542896', 'mp-117823

# Filtering the Results

This returns a lot of materials - 160 to be exact! In many cases, it is useful to filter down the number of materials. For example, we may want to exclude large unit cells that may be computationally intensive to study. Or we may want to restrict our results to only thermodynamically-stable oxides, by use of the material's [energy above hull](https://wiki.materialsproject.org/Glossary_of_Terms).

As a basic example, here we only keep the first 10 iron oxides that the Materials Project API returned to us, and discard the other 150.

In [4]:
#As a demonstration, take the first 10 iron oxides
some_iron_oxides = iron_oxides_ids[:10]
print(some_iron_oxides)

['mp-1271693', 'mp-136', 'mp-1271562', 'mp-1245108', 'mp-1096950', 'mp-1194030', 'mp-568345', 'mp-1271295', 'mp-13', 'mp-150']


# Bringing Materials Into the User Account

Now that we have filtered the results from Materials Project down to just 10 structures, we may want to study them further with the computational models provided by Mat3ra. For example, we may be interested in leveraging a DFT code to find the structure with the largest band-gap, or perhaps we want to conduct a high-throughput screening of each material's surface energies.

In [5]:
# Upload the first 10 iron oxides found to our account
exabyte_materials_api = MaterialEndpoints(*ENDPOINT_ARGS)
materials = exabyte_materials_api.import_from_materialsproject(MATERIALS_PROJECT_API_KEY, some_iron_oxides)

Finally, it is always useful to stay organized. Materials sets make this convenient, acting as a folder to keep a group of related materials in. This would be especially helpful if, in the future, we wanted run a calculation over all the oxides we found in this example.

In [6]:
# Move the iron oxides to a materials set, just for this example
materials_set = exabyte_materials_api.create_set({"name" : "Some Iron Oxides"})
for material in materials:
    exabyte_materials_api.move_to_set(material["_id"], "", materials_set["_id"])